In [1]:
from __future__ import print_function
import os
import neat
import json

import pandas as pd
import numpy as np
import random

import torch
import torch.nn as nn
import torch.optim as optim


from explaneat.core.backprop import NeatNet
from explaneat.core import backprop
from explaneat.core.backproppop import BackpropPopulation
from explaneat.visualization import visualize
from explaneat.core.experiment import ExperimentReporter
from explaneat.core.utility import one_hot_encode


from sklearn import datasets
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from copy import deepcopy

import time
from datetime import datetime


import gzip
try:
    import cPickle as pickle  # pylint: disable=import-error
except ImportError:
    import pickle  # pylint: disable=import-error

In [2]:

USE_CUDA = torch.cuda.is_available()
USE_CUDA = False
device = torch.device("cuda:1" if USE_CUDA else "cpu")


In [3]:
device

device(type='cpu')

# Iris Experiment

This experiment (a) test the experimental environment, but is also to evaluate the efficacy of the ExplaNEAT algorithm. Speed is a critical factor, as well as stability of results on population size. Total run time will also be measured

First, we need to set a random seed and a total stopping point in the number of generations

In [4]:
my_random_seed = 42
random.seed(my_random_seed)

In [5]:
def one_hot_encode(vals):
    width = max(vals)
    newVals = []
    for val in vals:
        blank = [0. for _ in range(width + 1)]
        blank[val] = 1.
        newVals.append(blank)
    return np.asarray(newVals)


## Dataset

We are going to work with the Iris dataset, which will be loaded from `sklearn`. We want to characterise the efficacy of the algorithm with regards to a mostly untransformed dataset, so we will only normalise the features

In [6]:
data = pd.read_csv('/Users/mike/dev-mtm/phd-neat-experiments/data/breast-cancer/breast-cancer.data', header=None)
data.columns = ['Class',
'age',
'menopause',
'tumor-size',
'inv-nodes',
'node-caps',
'deg-malig',
'breast',
'breast-quad',
'irradiat']


In [7]:
defs = {
    'Class': ['no-recurrence-events', 'recurrence-events'], 
    'age': ['10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-89', '90-99'],
    'menopause': ['lt40', 'ge40', 'premeno'], 
    'tumor-size': ['0-4', '5-9', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59'], 
    'inv-nodes': ['0-2', '3-5', '6-8', '9-11', '12-14', '15-17', '18-20', '21-23', '24-26', '27-29', '30-32', '33-35', '36-39'],
    'node-caps': ['yes', 'no', '?'], 
#     'deg-malig': ['0', '1', '2', '3', '?'],
    'breast': ['left', 'right'],
    'breast-quad': ['left_up', 'left_low', 'right_up', 'right_low', 'central', '?'],
    'irradiat': ['yes', 'no']
}

In [8]:
for category in defs:
    try:
        data[category] = data[category].apply(lambda x: defs[category].index(x))
    except ValueError:
        print(category)

In [9]:
x_cols = ['age',
'menopause',
'tumor-size',
'inv-nodes',
'node-caps',
'deg-malig',
'breast',
'breast-quad',
'irradiat']
y_col = 'Class'

In [10]:
# digits = datasets.load_digits()
# xs_raw = digits.data[:, :]
xs_raw = np.array(data[x_cols]).astype(np.float32)
scaler = StandardScaler()
scaler.fit(xs_raw)
xs = scaler.transform(xs_raw)
# ys = iris.target
ys = np.array(data[y_col]).astype(np.float32)
ys = np.array([[y] for y in ys])
# ys_onehot = one_hot_encode(ys)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(xs, ys, test_size=0.15, random_state=42)

In [12]:
xs_raw.shape

(286, 9)

In [13]:
# xs = torch.from_numpy(xs).to(device)
# ys = torch.from_numpy(ys).to(device)
X_train = torch.from_numpy(X_train).to(device)
X_test = torch.from_numpy(X_test).to(device)
y_train = torch.from_numpy(y_train).to(device)
y_test = torch.from_numpy(y_test).to(device)

In [14]:

y_train

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
      

Let's have a look at the data we are working with

In [15]:
xs[:5]

array([[-1.647779  ,  0.91447103,  0.53223157, -0.45661306,  0.37947333,
         1.2905643 , -0.9389244 , -0.14681807,  0.55850387],
       [-0.65772694,  0.91447103, -0.41913235, -0.45661306,  0.37947333,
        -0.0664261 ,  1.0650485 ,  0.6765147 ,  0.55850387],
       [-0.65772694,  0.91447103, -0.41913235, -0.45661306,  0.37947333,
        -0.0664261 , -0.9389244 , -0.14681807,  0.55850387],
       [ 1.3223774 , -0.91447103, -0.8948143 , -0.45661306,  0.37947333,
        -0.0664261 ,  1.0650485 , -0.9701508 ,  0.55850387],
       [-0.65772694,  0.91447103, -2.32186   , -0.45661306,  0.37947333,
        -0.0664261 ,  1.0650485 ,  1.4998473 ,  0.55850387]],
      dtype=float32)

In [16]:
ys[15:20]

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.]], dtype=float32)

In [17]:
y_test[:5]

tensor([[0.],
        [1.],
        [0.],
        [1.],
        [1.]])

## Performance metric

The NEAT implementation on which ExplaNEAT extends uses a single function call for evaluating fitness. Although this might be reworked for ExplaNEAT to be able to get consistency between the genome-evaluation and the backprop loss function, that can be reviewed later.

This use `CrossEntropyLoss` from `PyTorch`

In [18]:
def eval_genomes(genomes, config):
    loss = nn.BCELoss()
    loss = loss.to(device)
    for genome_id, genome in genomes.items():
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        preds = []
        for xi in xs:
            preds.append(net.activate(xi))
        genome.fitness = float(1./loss(torch.tensor(preds).to(device), torch.tensor(ys)))

## Base configuration

We are going to create the base configuration according to an external configuration file. Per experiment, we will adjust this, later, but this defines the defaults across all runs.

In [19]:
config_path = "./config-bchard"
base_config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     config_path)


We also want to put a hard limit on how long this can go on for.

In [20]:
maxNGenerations = 200

We will create a method to manage the instantiation of a population on the basis of a specific config.

In [21]:
def instantiate_population(config, xs, ys, saveLocation):

    if not os.path.exists(saveLocation):
        os.makedirs(saveLocation)
        
    config.save(os.path.join(saveLocation, 'config.conf'))

    # Create the population, which is the top-level object for a NEAT run.
    p = BackpropPopulation(config, 
                            xs, 
                            ys, 
                            criterion=nn.BCELoss())

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    p.add_reporter(neat.Checkpointer(5, filename_prefix=str(saveLocation) + "checkpoint-" ))
    bpReporter = backprop.BackpropReporter(True)
    p.add_reporter(bpReporter)
    p.add_reporter(ExperimentReporter(saveLocation))
    
    return p

# Experiment 1: Vary population size

The first experiment is going to examine the difference in run time different population sizes. 

In [22]:
epoch_points = [10, 25, 50, 100, 150]
# epoch_points = [10]

In [23]:
base_config.pop_size

50

In [24]:
saveLocationTemplate = './../../data/experiments/bchard/experiment-newalgo-{}-{}/'

## Start the experiment

In [25]:
xs

array([[-1.647779  ,  0.91447103,  0.53223157, ..., -0.9389244 ,
        -0.14681807,  0.55850387],
       [-0.65772694,  0.91447103, -0.41913235, ...,  1.0650485 ,
         0.6765147 ,  0.55850387],
       [-0.65772694,  0.91447103, -0.41913235, ..., -0.9389244 ,
        -0.14681807,  0.55850387],
       ...,
       [ 1.3223774 , -0.91447103, -0.41913235, ...,  1.0650485 ,
        -0.9701508 ,  0.55850387],
       [-0.65772694, -0.91447103,  0.53223157, ..., -0.9389244 ,
        -0.14681807,  0.55850387],
       [ 0.3323252 , -0.91447103,  0.53223157, ..., -0.9389244 ,
        -0.14681807,  0.55850387]], dtype=float32)

In [26]:
for epochs in epoch_points:
    for iteration_no in range(20):
        my_random_seed += 1
        random.seed(my_random_seed)
        start_time = datetime.now()
        
        print("################################################")
        print("################################################")
        print("Starting epochs {} iteration {}".format(epochs, iteration_no))
        print("Started at {}".format(start_time.strftime("%m/%d/%Y, %H:%M:%S")))
        print("################################################")
        print("################################################")
        
        
        config = deepcopy(base_config)
#         config.pop_size = pop_size
        
        saveLocation = saveLocationTemplate.format(epochs, iteration_no)
        
        p = instantiate_population(config, X_train, y_train, saveLocation)
        # Run for up to nGenerations generations.
        winner = p.run(eval_genomes, maxNGenerations, nEpochs = epochs)
        
        g = p.best_genome

        
        end_time = datetime.now()
        
        p.reporters.reporters[2].save_checkpoint(p.config, p.population, p.species, str(p.generation) + "-final")  
        
        winner_net = neat.nn.FeedForwardNetwork.create(winner, config)

        results = []
        for xi, xo in zip(xs, ys):
            output = winner_net.activate(xi)
            results.append([xi, xo, output])

        df = pd.DataFrame(results)
        df.to_csv(os.path.join(saveLocation, 'results.csv'))
        
        ancestry = p.reporters.reporters[3].trace_ancestry_of_species(g.key, p.reproduction.ancestors) 

        ancestors = {
            k: v['genome'] for k, v in p.reporters.reporters[3].ancestry.items()
        }
        
    
        
        ## Save all of these to disc
        filename = 'fullStatus.xplnt'
        print("Saving checkpoint to {0}".format(filename))

        with gzip.open(os.path.join(saveLocation, filename), 'w', compresslevel=5) as f:
            data = (p, g, ancestry, ancestors, random.getstate())
            pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)
            
            
        with gzip.open(os.path.join(saveLocation, 'train_test_data.pkl'), 'w', compresslevel=5) as f:
            train_Test = (X_train, X_test, y_train, y_test)
            pickle.dump(train_Test, f, protocol=pickle.HIGHEST_PROTOCOL)
#         visualize.create_ancestry_video(p.config, 
#                                         g, 
#                                         ancestry, 
#                                         ancestors, 
#                                         p.reporters.reporters[1], 
#                                         pathname=saveLocation)
        print("################################################")
        print("################################################")
        print("Have finished epochs {} iteration {}".format(epochs, iteration_no))
        print("Started at {}".format(start_time.strftime("%m/%d/%Y, %H:%M:%S")))
        print("The time is {}".format(end_time.strftime("%m/%d/%Y, %H:%M:%S")))
        print("################################################")
        print("################################################")
    

################################################
################################################
Starting epochs 10 iteration 0
Started at 03/27/2022, 18:11:25
################################################
################################################
The function - generationStart - has just started at 1648357885.6070862

 ****** Running generation 0 ****** 

The function - generationStart - took 0.00014472007751464844 seconds to complete
The function - pre_backprop - has just started at 1648357885.607241
The function - pre_backprop - took 5.412101745605469e-05 seconds to complete
The function - backprop - has just started at 1648357885.607309
about to start backprop with 10 epochs
expected scalar type Float but found Double
layer id: 1
layer input: 1
{'training': True, '_parameters': OrderedDict([('weight_1', Parameter containing:
tensor([[ 0.6889],
        [ 0.8759],
        [-0.8388],
        [-1.5640],
        [-1.1240],
        [ 0.3405],
        [ 1.0566],
        [ 0.127

KeyError: 1